In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/hourly-energy-consumption/est_hourly.paruqet
/kaggle/input/hourly-energy-consumption/DOM_hourly.csv
/kaggle/input/hourly-energy-consumption/EKPC_hourly.csv
/kaggle/input/hourly-energy-consumption/DUQ_hourly.csv
/kaggle/input/hourly-energy-consumption/DAYTON_hourly.csv
/kaggle/input/hourly-energy-consumption/PJME_hourly.csv
/kaggle/input/hourly-energy-consumption/PJM_Load_hourly.csv
/kaggle/input/hourly-energy-consumption/NI_hourly.csv
/kaggle/input/hourly-energy-consumption/FE_hourly.csv
/kaggle/input/hourly-energy-consumption/COMED_hourly.csv
/kaggle/input/hourly-energy-consumption/AEP_hourly.csv
/kaggle/input/hourly-energy-consumption/pjm_hourly_est.csv
/kaggle/input/hourly-energy-consumption/DEOK_hourly.csv
/kaggle/input/hourly-energy-consumption/PJMW_hourly.csv


In [2]:
!pip install --upgrade -q xgboost

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.9/115.9 MB 14.2 MB/s eta 0:00:0000:0100:01


In [3]:
import xgboost as xgb
import matplotlib.pyplot as plt
import seaborn as sns
# from sklearn.model_seletion import train_test_split

In [4]:
print("xgboost", xgb.__version__)

xgboost 3.1.2


### We know : Datetime is the comon column 
### What we need to do? : we have to use Datetime to join the all these csv files to create a single dataset

# Lets anylise all the CSV files for the basic understanding . such as 
* *what is the significance of this csv file.*
* *totol no of rows and column in each file.*
* *usefullness of the values in the dataset.*
* *if there are any missing values and what will be the impact of the missing values from that csv on the overall df for further analysis.*

# SO what I have understood so far is that 
we have data sitting in a foldar 
so we need to call all the files in this folder 
the files endswith .csv

In [5]:
import os

folder_path = "/kaggle/input/hourly-energy-consumption"

file_summary = {}

for file in os.listdir(folder_path):
    if file.endswith(".csv"):
        file_path = os.path.join(folder_path, file)
        df_temp = pd.read_csv(file_path)
        file_summary[file] = {"shape": df_temp.shape,
                             "columns": list(df_temp.columns),
            "dtypes": df_temp.dtypes.to_dict(),
            "missing_values": df_temp.isna().sum().to_dict()}

    # print(file_summary)

In [6]:
for file, summary in file_summary.items():
    print(f"\n==== {file} ====")
    print("Shape:", summary["shape"])
    print("Columns:", summary["columns"])
    print("Dtypes:", summary["dtypes"])
    print("Missing:", summary["missing_values"])


==== DOM_hourly.csv ====
Shape: (116189, 2)
Columns: ['Datetime', 'DOM_MW']
Dtypes: {'Datetime': dtype('O'), 'DOM_MW': dtype('float64')}
Missing: {'Datetime': 0, 'DOM_MW': 0}

==== EKPC_hourly.csv ====
Shape: (45334, 2)
Columns: ['Datetime', 'EKPC_MW']
Dtypes: {'Datetime': dtype('O'), 'EKPC_MW': dtype('float64')}
Missing: {'Datetime': 0, 'EKPC_MW': 0}

==== DUQ_hourly.csv ====
Shape: (119068, 2)
Columns: ['Datetime', 'DUQ_MW']
Dtypes: {'Datetime': dtype('O'), 'DUQ_MW': dtype('float64')}
Missing: {'Datetime': 0, 'DUQ_MW': 0}

==== DAYTON_hourly.csv ====
Shape: (121275, 2)
Columns: ['Datetime', 'DAYTON_MW']
Dtypes: {'Datetime': dtype('O'), 'DAYTON_MW': dtype('float64')}
Missing: {'Datetime': 0, 'DAYTON_MW': 0}

==== PJME_hourly.csv ====
Shape: (145366, 2)
Columns: ['Datetime', 'PJME_MW']
Dtypes: {'Datetime': dtype('O'), 'PJME_MW': dtype('float64')}
Missing: {'Datetime': 0, 'PJME_MW': 0}

==== PJM_Load_hourly.csv ====
Shape: (32896, 2)
Columns: ['Datetime', 'PJM_Load_MW']
Dtypes: {'Datet


# *From this output above that we have got we can see that there are 13 csv files* 
# *we dont need to join this dataset as his folder is a compilasion of various datasets* 
# *So we can take any csv file and build our pridiction model that file*


# So this was good to see that we did this analysis of various csv files in the main folder as it gave me the purpose to understand that all the files are many cites like Kansas, Penselvania, and others. Hence we need to perform our prediction analysis on the main column which is PJME_hourly.csv as this is the electrical department or the board that provides energy in the USA. So it would be better if we figure out the prediction for this company in the East as PJME -> PJM-East 

In [9]:
# df1 = pd.read_csv("../input/hourly-energy-consumption/AEP_hourly.csv")
# df2 = pd.read_csv("../input/hourly-energy-consumption/COMED_hourly.csv")
# df3 = pd.read_csv("../input/hourly-energy-consumption/DAYTON_hourly.csv")
# df4 = pd.read_csv("../input/hourly-energy-consumption/DEOK_hourly.csv")
# df5 = pd.read_csv("../input/hourly-energy-consumption/DOM_hourly.csv")
# df6 = pd.read_csv("../input/hourly-energy-consumption/DUQ_hourly.csv")
# df7 = pd.read_csv("../input/hourly-energy-consumption/EKPC_hourly.csv")
# df8 = pd.read_csv("../input/hourly-energy-consumption/FE_hourly.csv")
# df9 = pd.read_csv("../input/hourly-energy-consumption/NI_hourly.csv")
# df10 = pd.read_csv("../input/hourly-energy-consumption/PJME_hourly.csv")
# df11 = pd.read_csv("../input/hourly-energy-consumption/PJMW_hourly.csv")
# df12 = pd.read_csv("../input/hourly-energy-consumption/PJM_Load_hourly.csv")
# df13 = pd.read_csv("../input/hourly-energy-consumption/pjm_hourly_est.csv")

In [10]:
# def dataframe(df):
    
#         #we will perform a join 
#         # load the dataset as one data
        

In [11]:
# df_test1 = pd.merge(df1, df2)

In [12]:
# df_test2 = pd.merge(df3, df4)

In [13]:
# df_test3 = pd.merge(df5, df6)

In [14]:
# df_test4 = pd.merge(df7, df8)

In [15]:
# df_test5 = pd.merge(df9, df10)

In [16]:
# df_test6 = pd.merge(df11, df12)

In [17]:
# df_test7 = pd.merge(df_test1, df_test2)

In [18]:
# df_test8 = pd.merge(df_test3, df_test4)

In [19]:
# df_test9 = pd.merge(df_test5, df_test6)

In [20]:
# df_test10 = pd.merge(df_test7, df_test8)

In [21]:
# df_final = pd.merge(df_test9, df_test10)

In [22]:
# df = pd.merge(df_final , df13)